In [17]:
from data import RooftopDataset
from transforms import ToTensor
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
from torch import optim
import numpy as np
from engine import evaluate
import utils
from PIL import Image
from matplotlib import cm
from matplotlib.pyplot import imshow

In [2]:
#### REDRAW PICTURES WHEREBY EACH ROOF GETS ITS OWN COLOR #####
## ONLY NEEDS TO BE DONE ONCE ##
# data_root_path = 'images'
# dataset = RooftopDataset(root=data_root_path, label_subfolder='labels',
                         #grayscale=True
#                         )
# dataset.save_color_encoded_images()

In [3]:
dataset = RooftopDataset(root='images',
                         #grayscale=True,
                         transforms=ToTensor())

In [4]:
from torchvision.transforms import (
    # Satellite pictures have no orientation by nature
    RandomHorizontalFlip,
    RandomVerticalFlip,
    # Here RandomCrop might be better since all pictures are shot from similar distance
    RandomResizedCrop, 
    # Satellite pictures are invariant to translation and rotation. shear not so much. scale maybe but same satellite -> same distance to earth
    RandomAffine,
    RandomApply,
    Compose
)

In [24]:
#### DATASET AND DATALOADER INITIALIZATION ####
# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
transformations = [
    RandomAffine(degrees=(-180, 180), translate=(0.3, 0.3)),
    RandomHorizontalFlip(p=0.5),
    RandomVerticalFlip(p=0.5),
    RandomResizedCrop(256, scale=(0.7, 1.))
]
transformation = Compose(transformations)
dataset = RooftopDataset(root='images',
                         transforms=transformation,
                         #grayscale=to_grayscale
                        )
dataset_test = RooftopDataset(root='images', grayscale=False, transforms=None)

dataset = torch.utils.data.Subset(dataset, indices[:-2])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-2:])

data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=1,
        collate_fn=utils.collate_fn
)

data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=1,
        collate_fn=utils.collate_fn
)

In [14]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True, min_size=128)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
def train_one_epoch(model, optimizer, data_loader):,
    model.train()
    for i, (images, targets) in enumerate(data_loader):,
        targets = list(target for target in targets),
        loss_dict = model(images, targets)\n",
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

In [15]:
def train(model, dataloader, optimizer_name='sgd', nr_epochs=30):
    # train on the GPU or on the CPU, if a GPU is not available
    # device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    device = torch.device('cpu')
    # our dataset has two classes only - background and person
    num_classes = 2

    # construct an optimizer
    if optimizer_name == 'sgd':
        params = [p for p in model.parameters() if p.requires_grad]
        optimizer = torch.optim.SGD(params, lr=0.005,
                                    momentum=0.9, weight_decay=0.0005)
        # and a learning rate scheduler (NOT IMPLEMENTED ATM)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                       step_size=3,
                                                       gamma=0.1)
    elif optimizer_name == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(nr_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, dataloader)
        # update the learning rate
        lr_scheduler.step()
        
    print("That's it!")
    return model    

In [16]:
 # get the model using our helper function
model = get_model_instance_segmentation(num_classes)
model = train(model, data_loader)
# evaluate on the test dataset
# evaluate(model, data_loader_test, device=device)

{'loss_classifier': tensor(0.8630, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0.2134, grad_fn=<DivBackward0>), 'loss_mask': tensor(3.5333, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(0.1768, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(0.0160, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(0.3240, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0.2406, grad_fn=<DivBackward0>), 'loss_mask': tensor(1.3817, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(0.3955, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(0.3988, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(0.2596, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0.2375, grad_fn=<DivBackward0>), 'loss_mask': tensor(0.5867, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(0.3627, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(0.0298, grad_fn=

{'loss_classifier': tensor(0.1632, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0.1313, grad_fn=<DivBackward0>), 'loss_mask': tensor(0.5637, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(0.1379, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(0.0166, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(0.1671, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0.1639, grad_fn=<DivBackward0>), 'loss_mask': tensor(0.5372, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(0.1273, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(0.0226, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(0.1839, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0.1908, grad_fn=<DivBackward0>), 'loss_mask': tensor(0.4400, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(0.1746, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(0.0616, grad_fn=

{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(n

{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(n

{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(n

...

{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(n

{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(n

{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(n

{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(n

{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(nan, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(nan, grad_fn=<DivBackward0>), 'loss_mask': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(nan, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(nan, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(n

In [30]:
# pick one image from the test set
img, _ = dataset_test[1]
print(img.size())
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img])

torch.Size([3, 256, 256])


RuntimeError: cannot reshape tensor of 0 elements into shape [0, -1] because the unspecified dimension size -1 can be any value and is ambiguous

In [19]:
for row in img:
    

TypeError: __call__() missing 1 required positional argument: 'target'

In [ ]:
for i in range(len(dataset)):
    img, target = dataset[i]
    print('number rooftops:', len(target['labels']))
    print('img shape', img.size())

In [ ]:
print(image.shape)
im = Image.fromarray(image, 'RGB')

In [ ]:
img, target = dataset[0]
mask = target['masks']
image = np.array(img)
image = np.swapaxes(image, 0, 1)
image = np.swapaxes(image, 1, 2)
imshow(image)

In [ ]:
imshow(mask[2])